In [ ]:
from qiskit_nature.drivers import PySCFDriver,UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit_nature.transformers import FreezeCoreTransformer,ActiveSpaceTransformer,BaseTransformer
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit.aqua.operators import Z2Symmetries
from qiskit_nature.circuit.library import HartreeFock
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
from qiskit.algorithms import VQE
from IPython.display import display, clear_output
from qiskit import QuantumCircuit, transpile
from qiskit import Aer
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 
molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()
h1 = qmolecule.one_body_integrals
#print(h1)
h2 = qmolecule.two_body_integrals
#print(h2)
core=qmolecule.core_orbitals
print(core)
#******* list the fermionic operator with removing the orbital with the help of FreezcoreTransformer
problem = ElectronicStructureProblem(driver,[FreezeCoreTransformer(freeze_core=True,remove_orbitals=[4,3])])
#*********************************************************************************
second_q_ops = problem.second_q_ops()
#print(second_q_ops)
main_op = second_q_ops[0]
#print(main_op)
#***********************************************************************
#***********************************************************************
mapper_type = 'ParityMapper'
if mapper_type == 'ParityMapper':
    mapper = ParityMapper()
elif mapper_type == 'JordanWignerMapper':
    mapper = JordanWignerMapper()
elif mapper_type == 'BravyiKitaevMapper':
    mapper = BravyiKitaevMapper()
#-- Making convert and apply qubit reduction and z2symmetry_reduction
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True,z2symmetry_reduction=[1])
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
qubit_op = converter.convert(main_op, num_particles=num_particles)

#*****************************************************************
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print(init_state)
#******* Making a ansatz by using TwoLocal
# Single qubit rotations that are placed on all qubits with independent parameters
rotation_blocks = ['ry']
# Entangling gates
entanglement_blocks = 'cz'
# How the qubits are entangled 
l1 = [(0,2)]
l2=[(2,3)]
l3=[(2,1)]
# Repetitions of rotation_blocks + entanglement_blocks with independent parameters
repetitions = 3
# Skip the final rotation_blocks layer
skip_final_rotation_layer = False
ansatz = TwoLocal(qubit_op.num_qubits, rotation_blocks, entanglement_blocks, reps=repetitions, 
                      entanglement=[l1,l2,l3], skip_final_rotation_layer=skip_final_rotation_layer)
#*****************************************************************
backend = Aer.get_backend('statevector_simulator')
ansatz=transpile(ansatz,backend=backend,optimization_level=3)
#************************************************************************
optimizer_type = 'COBYLA'
# You may want to tune the parameters 
# of each optimizer, here the defaults are used
if optimizer_type == 'COBYLA':
    optimizer = COBYLA(maxiter=1000)
elif optimizer_type == 'L_BFGS_B':
    optimizer = L_BFGS_B(maxfun=500)
elif optimizer_type == 'SPSA':
    optimizer = SPSA(maxiter=2000)
elif optimizer_type == 'SLSQP':
    optimizer = SLSQP(maxiter=2000)
#************************************************************
#***********************************************************
def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []

# Set initial parameters of the ansatz
# We choose a fixed small displacement 
# So all participants start from similar starting point
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)
def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)

print(result)